# TP3: Classification binaire

In [66]:
# Importation
import nltk
import spacy
import sklearn
import re
import string
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [56]:
df = pd.read_csv("spam.csv")

In [57]:
df.head(5)

,label,text
0,ham,Peux-tu m'envoyer le document ?
1,ham,As-tu vu le dernier épisode de la série ?
2,spam,Augmentez votre revenu avec ce programme secret.
3,spam,Vous avez été sélectionné pour un cadeau gratuit.
4,ham,"J'ai terminé le rapport, je te l'envoie."


In [58]:
df.describe()

,label,text
count,100,100
unique,2,20
top,ham,Cliquez ici pour réclamer votre prix.
freq,50,11


In [59]:
# Initialisation
stop_words = set(stopwords.words('french')) 
lemmatizer = WordNetLemmatizer()

# Fonction de nettoyage
def nettoyer_texte(text):
    text = text.lower() 
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    words = text.split() 
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words] 
    return ' '.join(words)

# Appliquer le nettoyage
df['clean_text'] = df['text'].apply(nettoyer_texte)
df.head(5)

,label,text,clean_text
0,ham,Peux-tu m'envoyer le document ?,peuxtu menvoyer document
1,ham,As-tu vu le dernier épisode de la série ?,astu vu dernier épisode série
2,spam,Augmentez votre revenu avec ce programme secret.,augmentez revenu programme secret
3,spam,Vous avez été sélectionné pour un cadeau gratuit.,sélectionné cadeau gratuit
4,ham,"J'ai terminé le rapport, je te l'envoie.",jai terminé rapport lenvoie


In [60]:
df['label_num'] = df['label'].map({'ham': 0, 'spam': 1})
df.head(5)

,label,text,clean_text,label_num
0,ham,Peux-tu m'envoyer le document ?,peuxtu menvoyer document,0
1,ham,As-tu vu le dernier épisode de la série ?,astu vu dernier épisode série,0
2,spam,Augmentez votre revenu avec ce programme secret.,augmentez revenu programme secret,1
3,spam,Vous avez été sélectionné pour un cadeau gratuit.,sélectionné cadeau gratuit,1
4,ham,"J'ai terminé le rapport, je te l'envoie.",jai terminé rapport lenvoie,0


In [61]:
# Vectorisation TF-IDF
tfidf_vectorizer = TfidfVectorizer()
X = tfidf_vectorizer.fit_transform(df['text'])
y = df['label_num']

# Séparation train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [64]:
# Initialisation des modèles
models = {
    "Régression Logistique": LogisticRegression(max_iter=1000),
    "k-NN": KNeighborsClassifier(n_neighbors=5),
    "Naive Bayes": MultinomialNB()
}

In [65]:
# Validation croisée k=5
print("=== Validation croisée (5-fold) sur l'ensemble d'entraînement ===")
for name, model in models.items():
    scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
    print(f"{name} Accuracy: {scores.mean():.4f} (+/- {scores.std():.4f})")

=== Validation croisée (5-fold) sur l'ensemble d'entraînement ===
Régression Logistique Accuracy: 0.9750 (+/- 0.0500)
k-NN Accuracy: 0.9250 (+/- 0.0612)
Naive Bayes Accuracy: 0.9750 (+/- 0.0500)


In [67]:
print("\n=== Évaluation sur l'ensemble de test ===")
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(f"\n--- {name} ---")
    print("Classification Report:\n", classification_report(y_test, y_pred))
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


=== Évaluation sur l'ensemble de test ===

--- Régression Logistique ---
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        11
           1       1.00      1.00      1.00         9

    accuracy                           1.00        20
   macro avg       1.00      1.00      1.00        20
weighted avg       1.00      1.00      1.00        20

Confusion Matrix:
 [[11  0]
 [ 0  9]]

--- k-NN ---
Classification Report:
               precision    recall  f1-score   support

           0       0.92      1.00      0.96        11
           1       1.00      0.89      0.94         9

    accuracy                           0.95        20
   macro avg       0.96      0.94      0.95        20
weighted avg       0.95      0.95      0.95        20

Confusion Matrix:
 [[11  0]
 [ 1  8]]

--- Naive Bayes ---
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00 